In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import csv

In [2]:
import string
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import re

## Helpers

In [3]:
def show_na(raw):
    n = len(raw)
    for col in raw.columns:
        na_count = sum(raw[col].isna())
        print("| " + col + " | " + format(na_count, ",d") + " | " + "{:.1%}".format(na_count/n) + " |")

In [4]:
def show_nunique(raw):
    for c in ["country","designation","province","region_1","region_2","variety","winery"]:
        print(c + ": "+str(raw[c].nunique()))

In [5]:
def show_nunique2(raw):
    for c in ["country","designation","province","region_1","region_2","variety","winery","year"]:
        print(c + ": "+str(raw[c].nunique()))

In [6]:
def get_year(t):
    try:
        return [w for w in t.split() if re.search('^[0-9]{4}$', w)][0]
    except:
        return "unknown"

In [7]:
raw = pd.read_csv("winemag-data-130k-v2.csv")

In [8]:
raw["year"] = raw["title"].apply(get_year)

In [9]:
year = raw["year"]

In [10]:
#year.to_csv('years.csv')

In [11]:
show_na(raw)

| Unnamed: 0 | 0 | 0.0% |
| country | 63 | 0.0% |
| description | 0 | 0.0% |
| designation | 37,465 | 28.8% |
| points | 0 | 0.0% |
| price | 8,996 | 6.9% |
| province | 63 | 0.0% |
| region_1 | 21,247 | 16.3% |
| region_2 | 79,460 | 61.1% |
| taster_name | 26,244 | 20.2% |
| taster_twitter_handle | 31,213 | 24.0% |
| title | 0 | 0.0% |
| variety | 1 | 0.0% |
| winery | 0 | 0.0% |
| year | 0 | 0.0% |


| column | unique values | missing count | missing% | type |preprocessing |
| --|--|--|--| -- | -- |
| Unnamed: 0 | 0 | 0.0% | int| deleted |
| country | 43 | 63 | 0.0% | categorical | replace missing with "unknown" -> one-hot encoded |
| description | - | 0 | 0.0% | text | converted into bigram TF-IDF features | 
| designation | 35,777 | 37,465 | 28.8% | categorical |  replace missing with "unknown" -> one-hot encoded  |
| points | -| 0| 0.0% | real | - |
| price | - | 8,996 | 6.9% | real | removed missing row |
| province | 423 | 63 | 0.0% | categorical |  replace missing with "unknown" -> one-hot encoded  |
| region_1 | 1205 | 21,247 | 16.3% | categorical |  replace missing with "unknown" -> one-hot encoded  |
| region_2 | 18 | 79,460 | 61.1% | categorical |  replace missing with "unknown" -> one-hot encoded  |
| taster_name | - | 26,244 | 20.2% | text | deleted |
| taster_twitter_handle | - | 31,213 | 24.0% | text | deleted |
| title | - | 0 | 0.0% | text | extracted new column "year" |
| variety | 698 | 1 | 0.0% |  categorical | removed missing row |
| winery | 15855 | 0 | 0.0% |  categorical | one-hot encoded  |
| year | 89 | 0 | 0.0% |  categorical | new column -> one-hot encoded  

| column | unique values | missing count | missing% |
| --|--|--|--|
| Unnamed: 0 | 0 | 0.0% | int|
| country | 43 | 63 | 0.0% | categorical |
| description | - | 0 | 0.0% | text |
| designation | 35,777 | 37,465 | 28.8% | categorical |
| points | -| 0| 0.0% | real |
| price | - | 8,996 | 6.9% | real |
| province | 423 | 63 | 0.0% | categorical |
| region_1 | 1205 | 21,247 | 16.3% | categorical |
| region_2 | 18 | 79,460 | 61.1% | categorical |
| taster_name | - | 26,244 | 20.2% | text |
| taster_twitter_handle | - | 31,213 | 24.0% | text |
| title | - | 0 | 0.0% | text |
| variety | 698 | 1 | 0.0% |  categorical |
| winery | 15855 | 0 | 0.0% |  categorical |
| year | 89 | 0 | 0.0% |  categorical |

In [12]:
# del raw["Unnamed: 0"]
raw.drop(columns = ["Unnamed: 0","taster_name","taster_twitter_handle","title"], inplace=True)

In [13]:
# raw.info()
show_na(raw)

| country | 63 | 0.0% |
| description | 0 | 0.0% |
| designation | 37,465 | 28.8% |
| points | 0 | 0.0% |
| price | 8,996 | 6.9% |
| province | 63 | 0.0% |
| region_1 | 21,247 | 16.3% |
| region_2 | 79,460 | 61.1% |
| variety | 1 | 0.0% |
| winery | 0 | 0.0% |
| year | 0 | 0.0% |


In [14]:
# raw.dropna(subset = ['price'])

In [15]:
raw.dropna(axis = 0, subset = ["price"],inplace=True)

In [16]:
raw["province"].fillna(value = "UNK_province", inplace=True)
raw["region_1"].fillna(value = "UNK_region_1", inplace=True)
raw["region_2"].fillna(value = "UNK_region_2", inplace=True)
raw["variety"].fillna(value = "UNK_variety", inplace=True)

In [17]:
show_na(raw)

| country | 59 | 0.0% |
| description | 0 | 0.0% |
| designation | 34,779 | 28.7% |
| points | 0 | 0.0% |
| price | 0 | 0.0% |
| province | 0 | 0.0% |
| region_1 | 0 | 0.0% |
| region_2 | 0 | 0.0% |
| variety | 0 | 0.0% |
| winery | 0 | 0.0% |
| year | 0 | 0.0% |


In [18]:
show_nunique2(raw)

country: 42
designation: 35776
province: 423
region_1: 1205
region_2: 18
variety: 698
winery: 15855
year: 89


In [36]:
countries = pd.unique(raw["country"])

#for c in countries:
    #print(c + ": " + str(len(raw[raw["country"] == c])))
    #print(c)
    #print(len(raw[raw["country"] == c]))

In [20]:
US_wines = raw[raw["country"] == "US"]

In [21]:
US_wines

,country,description,designation,points,price,province,region_1,region_2,variety,winery,year
2,US,"Tart and snappy, the flavors of lime flesh and...",NaN,87,14.0,Oregon,Willamette Valley,Willamette Valley,Pinot Gris,Rainstorm,2013
3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,UNK_region_2,Riesling,St. Julian,2013
4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Willamette Valley,Pinot Noir,Sweet Cheeks,2012
10,US,"Soft, supple plum envelopes an oaky structure ...",Mountain Cuvée,87,19.0,California,Napa Valley,Napa,Cabernet Sauvignon,Kirkland Signature,2011
12,US,"Slightly reduced, this wine offers a chalky, t...",NaN,87,34.0,California,Alexander Valley,Sonoma,Cabernet Sauvignon,Louis M. Martini,2012
14,US,Building on 150 years and six generations of w...,NaN,87,12.0,California,Central Coast,Central Coast,Chardonnay,Mirassou,2012
19,US,"Red fruit aromas pervade on the nose, with cig...",NaN,87,32.0,Virginia,Virginia,UNK_region_2,Meritage,Quiévremont,2012
20,US,Ripe aromas of dark berries mingle with ample ...,Vin de Maison,87,23.0,Virginia,Virginia,UNK_region_2,Red Blend,Quiévremont,2012
21,US,"A sleek mix of tart berry, stem and herb, alon...",NaN,87,20.0,Oregon,Oregon,Oregon Other,Pinot Noir,Acrobat,2013
23,US,This wine from the Geneseo district offers aro...,Signature Selection,87,22.0,California,Paso Robles,Central Coast,Merlot,Bianchi,2011


In [69]:
US_wines.to_csv('USwines.csv')

# output csv

In [ ]:
raw.drop(columns = ["title","description"], inplace=True)

In [ ]:
raw.to_csv('wines.csv')

In [ ]:
raw.shape

In [ ]:
#raw = raw.iloc[1:20000,:]

In [ ]:
raw = raw[raw["price"]<300]

In [ ]:
show_nunique2(raw)

In [ ]:
country = pd.get_dummies(raw["country"])
designation = pd.get_dummies(raw["designation"])
province = pd.get_dummies(raw["province"])
region_1 = pd.get_dummies(raw["region_1"])
region_2 = pd.get_dummies(raw["region_2"])
variety = pd.get_dummies(raw["variety"])
winery = pd.get_dummies(raw["winery"])
points = pd.DataFrame(raw["points"])
price = pd.DataFrame(raw["price"])

year = pd.get_dummies(raw["year"])

In [ ]:
from sklearn.preprocessing import OneHotEncoder
onehotencoder = OneHotEncoder()

In [ ]:
X_cat = onehotencoder.fit_transform(raw[["region_1","variety","year","winery"]]).toarray()

### Description: TF-IDF

In [22]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [23]:
from sklearn.feature_extraction import text

In [24]:
#text.ENGLISH_STOP_WORDS

In [25]:
vectorizer = TfidfVectorizer(ngram_range = (2,2),
                            stop_words = 'english',
                             lowercase = True)
# max_features
# norm = l2 by default

In [26]:
desc = vectorizer.fit_transform(US_wines["description"])

In [27]:
len(vectorizer.vocabulary_)

423314

In [28]:
tf_idf = (desc.toarray()) 
tf_idf.shape

(54265, 423314)

In [ ]:
# print("\n\nScores : \n", tf_idf) 

In [ ]:
type(tf_idf)

In [18]:
TFIDF = pd.DataFrame(tf_idf)

In [19]:
TFIDF.shape

(120975, 738234)

In [20]:
TFIDF.head(1)

,0,1,2,3,4,5,6,7,8,9,...,738224,738225,738226,738227,738228,738229,738230,738231,738232,738233
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [21]:
TFIDF.to_csv('TFIDF.csv')

KeyboardInterrupt: 